# NER - ConNER

Due to dependency issues, I'm running this in a Docker container

Steps for running in Docker environment
1. Install
```
sudo apt update
sudo apt install docker.io
```

2. Create the Dockerfile (already created)

3. Build Docker image
```
docker build -t conner .
```

4. Run container
```
docker run -it -p 8888:8888 -v /ner_ConNER_woojae.ipynb conner bash
```

5. Start Jupyter notebook in the workspace, paste in the token in terminal 
```
jupyter notebook --ip 0.0.0.0 --no-browser --allow-root
```

Notebook should save itself as long as the container is run in the same directory (ner_ConNER_woojae.ipynb) but please download the latest notebook and push it to the project repo for everyone else to see. 

## Data Setup

In [1]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 46.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 35.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# Libraries 

import sys
import os
from pathlib import Path

import pandas as pd 
import numpy as np

import ast 

from collections import Counter

In [3]:
# Global settings

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [6]:
# Path for datasets

#datapath = '../data/'
datapath = './'

In [8]:
# Load datasets

df_train = pd.read_csv(f'{datapath}' + 'OfficialTrainingSet1.csv')
df_val = pd.read_csv(f'{datapath}' + 'OfficialValidationSet1.csv')
df_test = pd.read_csv(f'{datapath}' + 'OfficialTestSet1.csv')

print("Shape of train dataset:", df_train.shape)
print("Shape of validation dataset:", df_val.shape)
print("Shape of test dataset:", df_test.shape)

df_train.head(3)

Shape of train dataset: (500, 13)
Shape of validation dataset: (500, 13)
Shape of test dataset: (500, 13)


,article_code,title,abstract,chemicals,diseases,chemical_start_indices,chemical_end_indices,disease_start_indices,disease_end_indices,chemical_ids,disease_ids,CID_chemical,CID_disease
0,227508,Naloxone reverses the antihypertensive effect ...,"In unanesthetized, spontaneously hypertensive ...","['Naloxone', 'clonidine', 'clonidine', 'nalozo...","['hypertensive', 'hypotensive', 'hypertensive'...","['0', '49', '181', '244', '306', '354', '364',...","['8', '58', '190', '252', '322', '362', '372',...","['93', '274', '469', '750']","['105', '285', '481', '762']","['D009270', 'D003000', 'D003000', '-1', 'D0087...","['D006973', 'D007022', 'D006973', 'D006973']",['D008750'],['D007022']
1,354896,Lidocaine-induced cardiac asystole.,Intravenous administration of a single 50-mg b...,"['Lidocaine', 'lidocaine', 'lidocaine']","['cardiac asystole', 'depression', 'bradyarrhy...","['0', '90', '409']","['9', '99', '418']","['18', '142', '331']","['34', '152', '347']","['D008012', 'D008012', 'D008012']","['D006323', 'D003866', 'D001919']",['D008012'],['D006323']
2,435349,Suxamethonium infusion rate and observed fasci...,Suxamethonium chloride (Sch) was administered ...,"['Suxamethonium', 'Suxamethonium chloride', 'S...","['fasciculations', 'tetanic', 'Fasciculations'...","['0', '80', '104', '312']","['13', '102', '107', '315']","['41', '265', '395', '483', '523', '538', '561...","['55', '272', '409', '496', '536', '544', '568...","['D013390', 'D013390', 'D013390', 'D013390']","['D005207', 'D013746', 'D005207', 'D005207', '...",['D013390'],['D005207']


In [9]:
# Data transformation functions

def convert_col_to_list(string):
    """
    Converts all string columns that look like lists (col index 3 to end) into actual lists 
    """
    return ast.literal_eval(string)


def lowercase_cols(lst):
    """
    Converts chemicals and diseases column to lowercase
    """
    return [item.lower() for item in lst]


def map_cid_to_chemical_name(row):
    """
    Maps CID of chemical in the CID_chemical column into the actual name of the chemical
    """
    cid_chemicals = row['CID_chemical']
    chemical_ids = row['chemical_ids']
    chemicals = row['chemicals']
    
    chemical_names = []
    
    for cid in cid_chemicals:
        if cid in chemical_ids:
            idx = chemical_ids.index(cid)
            chemical_names.append(chemicals[idx])
        else:
            chemical_names.append('unknown')
    
    return chemical_names


def map_cid_to_disease_name(row):
    """
    Maps CID of disease in the CID_disease column into the actual name of the disease
    """
    cid_diseases = row['CID_disease']
    disease_ids = row['disease_ids']
    diseases = row['diseases']
    
    disease_names = []
    
    for cid in cid_diseases:
        if cid in disease_ids:
            idx = disease_ids.index(cid) 
            disease_names.append(diseases[idx]) 
        else:
            disease_names.append('unknown')
    
    return disease_names


# Function to handle "unknown" for chemical names
def map_cid_to_chemical_name_unknown(data):
    '''
    Addresses 'unknown' instances of CID_chemical_names caused by chemicals with pipe (|) notation
    '''
    chemical_id_map = {}
    for i, row in data.iterrows():
        for cid, chemical in zip(row['chemical_ids'], row['chemicals']):
            chemical_id_map[cid] = chemical
    
    # Function to map "unknown" to the correct chemical name if possible
    def resolve_unknown_chemical_name(cids):
        names = []
        for cid in cids:
            # Split combined IDs (separated by '|') and check for matches in the map
            split_ids = cid.split('|')
            name = ' | '.join([chemical_id_map.get(split_id, 'unknown') for split_id in split_ids])
            names.append(name)
        return names

    # Apply the function only to rows where CID_chemical_name has "unknown"
    data['CID_chemical_name'] = data.apply(lambda row: resolve_unknown_chemical_name(row['CID_chemical']) 
                                       if 'unknown' in row['CID_chemical_name'] else row['CID_chemical_name'], axis=1)
    return data

# Function to handle "Unknown" for disease names
def map_cid_to_disease_name_unknown(data):
    '''
    Addresses 'unknown' instances of CID_disease_names caused by diseases with pipe (|) notation
    '''
    disease_id_map = {}
    for i, row in data.iterrows():
        for cid, disease in zip(row['disease_ids'], row['diseases']):
            disease_id_map[cid] = disease
    
    # Function to map "unknown" to the correct disease name if possible
    def resolve_unknown_disease_name(cids):
        names = []
        for cid in cids:
            # Split combined IDs (separated by '|') and check for matches in the map
            split_ids = cid.split('|')
            name = ' | '.join([disease_id_map.get(split_id, 'unknown') for split_id in split_ids])
            names.append(name)
        return names

    # Apply the function only to rows where CID_disease_name has "Unknown"
    data['CID_disease_name'] = data.apply(lambda row: resolve_unknown_disease_name(row['CID_disease']) 
                                      if 'unknown' in row['CID_disease_name'] else row['CID_disease_name'], axis=1)
    return data

In [10]:
# Apply the data transformations functions to all three datasets

list_columns = ['chemicals', 'diseases', 'chemical_ids', 'disease_ids', 'CID_chemical', 'CID_disease']
for col in list_columns:
    df_train[col] = df_train[col].apply(convert_col_to_list) 
    df_val[col] = df_val[col].apply(convert_col_to_list) 
    df_test[col] = df_test[col].apply(convert_col_to_list) 

df_train['chemicals'] = df_train['chemicals'].apply(lowercase_cols)
df_train['diseases'] = df_train['diseases'].apply(lowercase_cols)
df_val['chemicals'] = df_val['chemicals'].apply(lowercase_cols)
df_val['diseases'] = df_val['diseases'].apply(lowercase_cols)
df_test['chemicals'] = df_test['chemicals'].apply(lowercase_cols)
df_test['diseases'] = df_test['diseases'].apply(lowercase_cols)

df_train['CID_chemical_name'] = df_train.apply(map_cid_to_chemical_name, axis=1)
df_train['CID_disease_name'] = df_train.apply(map_cid_to_disease_name, axis=1)
df_val['CID_chemical_name'] = df_val.apply(map_cid_to_chemical_name, axis=1)
df_val['CID_disease_name'] = df_val.apply(map_cid_to_disease_name, axis=1)
df_test['CID_chemical_name'] = df_test.apply(map_cid_to_chemical_name, axis=1)
df_test['CID_disease_name'] = df_test.apply(map_cid_to_disease_name, axis=1)

df_train = map_cid_to_chemical_name_unknown(df_train)
df_train = map_cid_to_disease_name_unknown(df_train)
df_val = map_cid_to_chemical_name_unknown(df_val)
df_val = map_cid_to_disease_name_unknown(df_val)
df_test = map_cid_to_chemical_name_unknown(df_test)
df_test = map_cid_to_disease_name_unknown(df_test)

df_train.head(3)

,article_code,title,abstract,chemicals,diseases,chemical_start_indices,chemical_end_indices,disease_start_indices,disease_end_indices,chemical_ids,disease_ids,CID_chemical,CID_disease,CID_chemical_name,CID_disease_name
0,227508,Naloxone reverses the antihypertensive effect ...,"In unanesthetized, spontaneously hypertensive ...","[naloxone, clonidine, clonidine, nalozone, alp...","[hypertensive, hypotensive, hypertensive, hype...","['0', '49', '181', '244', '306', '354', '364',...","['8', '58', '190', '252', '322', '362', '372',...","['93', '274', '469', '750']","['105', '285', '481', '762']","[D009270, D003000, D003000, -1, D008750, D0092...","[D006973, D007022, D006973, D006973]",[D008750],[D007022],[alpha-methyldopa],[hypotensive]
1,354896,Lidocaine-induced cardiac asystole.,Intravenous administration of a single 50-mg b...,"[lidocaine, lidocaine, lidocaine]","[cardiac asystole, depression, bradyarrhythmias]","['0', '90', '409']","['9', '99', '418']","['18', '142', '331']","['34', '152', '347']","[D008012, D008012, D008012]","[D006323, D003866, D001919]",[D008012],[D006323],[lidocaine],[cardiac asystole]
2,435349,Suxamethonium infusion rate and observed fasci...,Suxamethonium chloride (Sch) was administered ...,"[suxamethonium, suxamethonium chloride, sch, sch]","[fasciculations, tetanic, fasciculations, fasc...","['0', '80', '104', '312']","['13', '102', '107', '315']","['41', '265', '395', '483', '523', '538', '561...","['55', '272', '409', '496', '536', '544', '568...","[D013390, D013390, D013390, D013390]","[D005207, D013746, D005207, D005207, D005207, ...",[D013390],[D005207],[suxamethonium],[fasciculations]


## ConNER

In [11]:
!pip install transformers torch


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [12]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

import json
from pprint import pprint

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained('./bc5cdr')
model = AutoModelForTokenClassification.from_pretrained('./bc5cdr')
print(model.config)

RobertaConfig {
  "architectures": [
    "RobertaForTokenClassification_v2"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 50008
}



In [14]:
# Example 1

example_data = {'text': ['Test results show that Aspirin was effective at lowering pain among patients that are diagnosed with diabetes']}
example_df = pd.DataFrame(example_data)

example_ner_pipeline = pipeline('ner', model=model, tokenizer=tokenizer)

example_df['entities'] = example_df['text'].apply(lambda x: example_ner_pipeline(x))

pd.set_option('display.max_colwidth', None)
pprint(example_df['entities'][0])

[{'entity': 'LABEL_0', 'score': 0.9999155402183533, 'word': '<s>'},
 {'entity': 'LABEL_0', 'score': 0.999402642250061, 'word': 'ĠTest'},
 {'entity': 'LABEL_0', 'score': 0.9999911189079285, 'word': 'Ġresults'},
 {'entity': 'LABEL_0', 'score': 0.9999900460243225, 'word': 'Ġshow'},
 {'entity': 'LABEL_0', 'score': 0.9999949932098389, 'word': 'Ġthat'},
 {'entity': 'LABEL_1', 'score': 0.9999805688858032, 'word': 'ĠAs'},
 {'entity': 'LABEL_3', 'score': 0.9998977780342102, 'word': 'pirin'},
 {'entity': 'LABEL_0', 'score': 0.9999927878379822, 'word': 'Ġwas'},
 {'entity': 'LABEL_0', 'score': 0.9999933242797852, 'word': 'Ġeffective'},
 {'entity': 'LABEL_0', 'score': 0.9999872446060181, 'word': 'Ġat'},
 {'entity': 'LABEL_0', 'score': 0.9999865889549255, 'word': 'Ġlowering'},
 {'entity': 'LABEL_2', 'score': 0.9999061226844788, 'word': 'Ġpain'},
 {'entity': 'LABEL_0', 'score': 0.9999793767929077, 'word': 'Ġamong'},
 {'entity': 'LABEL_0', 'score': 0.999994158744812, 'word': 'Ġpatients'},
 {'entity': 

In [15]:
# Example 2 (actual abstract (train row 0))

example_data = {'text': ['In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10(-8) to 10(-5) M, did not influence stereoselective binding of [3H]-naloxone (8 nM), and naloxone, 10(-8) to 10(-4) M, did not influence clonidine-suppressible binding of [3H]-dihydroergocryptine (1 nM). These findings indicate that in spontaneously hypertensive rats the effects of central alpha-adrenoceptor stimulation involve activation of opiate receptors. As naloxone and clonidine do not appear to interact with the same receptor site, the observed functional antagonism suggests the release of an endogenous opiate by clonidine or alpha-methyldopa and the possible role of the opiate in the central control of sympathetic tone.']}
example_df = pd.DataFrame(example_data)

example_ner_pipeline = pipeline('ner', model=model, tokenizer=tokenizer)

example_df['entities'] = example_df['text'].apply(lambda x: example_ner_pipeline(x))

pd.set_option('display.max_colwidth', None)
pprint(example_df['entities'][0])

[{'entity': 'LABEL_0', 'score': 0.99994295835495, 'word': '<s>'},
 {'entity': 'LABEL_0', 'score': 0.9999759197235107, 'word': 'ĠIn'},
 {'entity': 'LABEL_0', 'score': 0.9999955892562866, 'word': 'Ġun'},
 {'entity': 'LABEL_0', 'score': 0.9999971985816956, 'word': 'anesthetized'},
 {'entity': 'LABEL_0', 'score': 0.9999886155128479, 'word': ','},
 {'entity': 'LABEL_0', 'score': 0.9912057518959045, 'word': 'Ġspontaneously'},
 {'entity': 'LABEL_2', 'score': 0.8342937231063843, 'word': 'Ġhypertensive'},
 {'entity': 'LABEL_0', 'score': 0.9999905228614807, 'word': 'Ġrats'},
 {'entity': 'LABEL_0', 'score': 0.9999895095825195, 'word': 'Ġthe'},
 {'entity': 'LABEL_0', 'score': 0.9977853894233704, 'word': 'Ġdecrease'},
 {'entity': 'LABEL_0', 'score': 0.9951260685920715, 'word': 'Ġin'},
 {'entity': 'LABEL_0', 'score': 0.9985578656196594, 'word': 'Ġblood'},
 {'entity': 'LABEL_0', 'score': 0.9991630911827087, 'word': 'Ġpressure'},
 {'entity': 'LABEL_0', 'score': 0.9990352988243103, 'word': 'Ġand'},
 {'

In [16]:
# Example 3 (actual abstract (train row 8))

example_data = {'text': [
'During an 18-month period of study 41 hemodialyzed patients receiving desferrioxamine (10-40 mg/kg BW/3 times weekly) for the first time were monitored for detection of audiovisual toxicity. 6 patients presented clinical symptoms of visual or auditory toxicity. Moreover, detailed ophthalmologic and audiologic studies disclosed abnormalities in 7 more asymptomatic patients. Visual toxicity was of retinal origin and was characterized by a tritan-type dyschromatopsy, sometimes associated with a loss of visual acuity and pigmentary retinal deposits. Auditory toxicity was characterized by a mid- to high-frequency neurosensorial hearing loss and the lesion was of the cochlear type. Desferrioxamine withdrawal resulted in a complete recovery of visual function in 1 patient and partial recovery in 3, and a complete reversal of hearing loss in 3 patients and partial recovery in 3. This toxicity appeared in patients receiving the higher doses of desferrioxamine or coincided with the normalization of ferritin or aluminium serum levels. The data indicate that audiovisual toxicity is not an infrequent complication in hemodialyzed patients receiving desferrioxamine. Periodical audiovisual monitoring should be performed on hemodialyzed patients receiving the drug in order to detect adverse effects as early as possible.'
]}
example_df = pd.DataFrame(example_data)

example_ner_pipeline = pipeline('ner', model=model, tokenizer=tokenizer)

example_df['entities'] = example_df['text'].apply(lambda x: example_ner_pipeline(x))

pd.set_option('display.max_colwidth', None)
pprint(example_df['entities'][0])

[{'entity': 'LABEL_0', 'score': 0.9998809099197388, 'word': '<s>'},
 {'entity': 'LABEL_0', 'score': 0.9997475743293762, 'word': 'ĠDuring'},
 {'entity': 'LABEL_0', 'score': 0.9999973177909851, 'word': 'Ġan'},
 {'entity': 'LABEL_0', 'score': 0.9999976754188538, 'word': 'Ġ18'},
 {'entity': 'LABEL_0', 'score': 0.9999966025352478, 'word': '-'},
 {'entity': 'LABEL_0', 'score': 0.9999979734420776, 'word': 'month'},
 {'entity': 'LABEL_0', 'score': 0.9999955892562866, 'word': 'Ġperiod'},
 {'entity': 'LABEL_0', 'score': 0.9999957084655762, 'word': 'Ġof'},
 {'entity': 'LABEL_0', 'score': 0.999993085861206, 'word': 'Ġstudy'},
 {'entity': 'LABEL_0', 'score': 0.9999963641166687, 'word': 'Ġ41'},
 {'entity': 'LABEL_0', 'score': 0.9999946355819702, 'word': 'Ġhem'},
 {'entity': 'LABEL_0', 'score': 0.9999964237213135, 'word': 'odial'},
 {'entity': 'LABEL_0', 'score': 0.9999974966049194, 'word': 'yzed'},
 {'entity': 'LABEL_0', 'score': 0.9999967217445374, 'word': 'Ġpatients'},
 {'entity': 'LABEL_0', 'scor

Where is the mapping for LABEL_0, LABEL_1, etc.?

- LABEL_0 - Non-entity
- LABEL_1 - Chemical
- LABEL_2 - Disease
- LABEL_3 - Chemical subword tokens
- LABEL_4 - Disease subword tokens

## ConNER Processing & Evaluation

In [44]:
ner_pipeline = pipeline('ner', model=model, tokenizer=tokenizer)

df_train_eval = df_train.copy()

# Get around IndexError by truncating to max_length
max_length=512
def process_abstract_with_ner(abstract):
    tokens = tokenizer.tokenize(abstract)
    if len(tokens) > max_length:
        # Skip abstracts that are too long
        print(f"Skipping abstract with tokens over {max_length} limit.")
        return []  # Return an empty list for skipped abstracts
    else:
        return ner_pipeline(abstract)

df_train_eval['predicted_entities'] = df_train_eval['abstract'].apply(process_abstract_with_ner)

print(df_train_eval[['abstract', 'chemicals', 'diseases', 'predicted_entities']].head(3))

Skipping abstract with tokens over 512 limit.
Skipping abstract with tokens over 512 limit.
Skipping abstract with tokens over 512 limit.
Skipping abstract with tokens over 512 limit.
Skipping abstract with tokens over 512 limit.
Skipping abstract with tokens over 512 limit.
Skipping abstract with tokens over 512 limit.
Skipping abstract with tokens over 512 limit.
Skipping abstract with tokens over 512 limit.
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [45]:
def extract_chemicals_and_diseases(entities):
    chemicals = []
    diseases = []
    
    current_chemical = ''
    current_disease = ''
    
    for entity in entities:
        if entity['entity'] == 'LABEL_1':
            if current_chemical:
                chemicals.append(current_chemical.strip())
                current_chemical = ''
            current_chemical += entity['word'].strip()
        elif entity['entity'] == 'LABEL_2':
            if current_disease:
                diseases.append(current_disease.strip())
                current_disease = ''
            current_disease += entity['word'].strip()
        elif entity['entity'] == 'LABEL_3':
            current_chemical += entity['word'].strip()
        elif entity['entity'] == 'LABEL_4':
            current_disease += entity['word'].strip()

    if current_chemical:
        chemicals.append(current_chemical.strip())
    if current_disease:
        diseases.append(current_disease.strip())
    
    return chemicals, diseases

In [46]:
# Apply the updated function to your predicted entities
df_train_eval[['predicted_chemicals', 'predicted_diseases']] = df_train_eval['predicted_entities'].apply(
    lambda entities: pd.Series(extract_chemicals_and_diseases(entities))
)

print(df_train_eval[['abstract', 'chemicals', 'diseases', 'predicted_chemicals', 'predicted_diseases']].head())

In [48]:
df_train_eval[['abstract', 'chemicals', 'diseases', 'predicted_chemicals', 'predicted_diseases']].head()

,abstract,chemicals,diseases,predicted_chemicals,predicted_diseases
0,"In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10(-8) to 10(-5) M, did not influence stereoselective binding of [3H]-naloxone (8 nM), and naloxone, 10(-8) to 10(-4) M, did not influence clonidine-suppressible binding of [3H]-dihydroergocryptine (1 nM). These findings indicate that in spontaneously hypertensive rats the effects of central alpha-adrenoceptor stimulation involve activation of opiate receptors. As naloxone and clonidine do not appear to interact with the same receptor site, the observed functional antagonism suggests the release of an endogenous opiate by clonidine or alpha-methyldopa and the possible role of the opiate in the central control of sympathetic tone.","[naloxone, clonidine, clonidine, nalozone, alpha-methyldopa, naloxone, naloxone, clonidine, 3h-naloxone, naloxone, clonidine, 3h-dihydroergocryptine, naloxone, clonidine, clonidine, alpha-methyldopa]","[hypertensive, hypotensive, hypertensive, hypertensive]","[Ġclonidine, Ġnalozone, Ġalpha-methyldopa, Ġnaloxone, ĠNaloxone, Ġclonidine, Ġ[3H]-naloxone, Ġnaloxone, Ġclonidine, Ġ[3H]-dihydroergocryptine, Ġnaloxone, Ġclonidine, Ġclonidine, Ġalpha-methyldopa]","[Ġhypertensive, Ġhypotensive, Ġhypertensive, Ġhypertensive]"
1,"Intravenous administration of a single 50-mg bolus of lidocaine in a 67-year-old man resulted in profound depression of the activity of the sinoatrial and atrioventricular nodal pacemakers. The patient had no apparent associated conditions which might have predisposed him to the development of bradyarrhythmias; and, thus, this probably represented a true idiosyncrasy to lidocaine.","[lidocaine, lidocaine, lidocaine]","[cardiac asystole, depression, bradyarrhythmias]","[Ġlidocaine, Ġlidocaine]","[Ġdepression, Ġbradyarrhythmias]"
2,"Suxamethonium chloride (Sch) was administered i.v. to 36 adult males at six rates: 0.25 mg s-1 to 20 mg s-1. The infusion was discontinued either when there was no muscular response to tetanic stimulation of the ulnar nerve or when Sch 120 mg was exceeded. Six additional patients received a 30-mg i.v. bolus dose. Fasciculations in six areas of the body were scored from 0 to 3 and summated as a total fasciculation score. The times to first fasciculation, twitch suppression and tetanus suppression were inversely related to the infusion rates. Fasciculations in the six areas and the total fasciculation score were related directly to the rate of infusion. Total fasciculation scores in the 30-mg bolus group and the 5-mg s-1 and 20-mg s-1 infusion groups were not significantly different.","[suxamethonium, suxamethonium chloride, sch, sch]","[fasciculations, tetanic, fasciculations, fasciculation, fasciculation, twitch, tetanus, fasciculations, fasciculation, fasciculation]","[ĠSuxamethoniumĠchloride, Sch, ĠSch]","[Ġtetanic, ĠFasciculations, Ġfasciculation, Ġfasciculation, Ġtwitch, Ġtetanus, ĠFasciculations, Ġfasciculation, Ġfasciculation]"
3,"Galanthamine hydrobromide, an anticholinesterase drug capable of penetrating the blood-brain barrier, was used in a patient demonstrating central effects of scopolamine (hyoscine) overdosage. It is longer acting than physostigmine and is used in anaesthesia to reverse the non-depolarizing neuromuscular block. However, studies into the dose necessary to combating scopolamine intoxication are indicated.","[galanthamine hydrobromide, scopolamine, hyoscine, galanthamine hydrobromide, scopolamine, hyoscine, physostigmine, scopolamine]",[overdosage],"[ĠGalanthamineĠhydrobromide, Ġscopolamine, hyoscine, Ġphysostigmine, Ġscopolamine]",[

In [50]:
# Remove Ġ token
df_train_eval['predicted_chemicals'] = df_train_eval['predicted_chemicals'].apply(lambda x: [chem.replace('Ġ', '').strip() for chem in x])
df_train_eval['predicted_diseases'] = df_train_eval['predicted_diseases'].apply(lambda x: [chem.replace('Ġ', '').strip() for chem in x])

# Remove bracket notation
df_train_eval['predicted_chemicals'] = df_train_eval['predicted_chemicals'].apply(lambda x: [chem.replace('[', '').replace(']', '').strip() for chem in x])
df_train_eval['predicted_diseases'] = df_train_eval['predicted_diseases'].apply(lambda x: [chem.replace('[', '').replace(']', '').strip() for chem in x])

# lowercase
df_train_eval['predicted_chemicals'] = df_train_eval['predicted_chemicals'].apply(lambda x: [chem.lower().strip() for chem in x])
df_train_eval['predicted_diseases'] = df_train_eval['predicted_diseases'].apply(lambda x: [chem.lower().strip() for chem in x])

df_train_eval[['abstract', 'chemicals', 'diseases', 'predicted_chemicals', 'predicted_diseases']].head(3)

,abstract,chemicals,diseases,predicted_chemicals,predicted_diseases
0,"In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10(-8) to 10(-5) M, did not influence stereoselective binding of [3H]-naloxone (8 nM), and naloxone, 10(-8) to 10(-4) M, did not influence clonidine-suppressible binding of [3H]-dihydroergocryptine (1 nM). These findings indicate that in spontaneously hypertensive rats the effects of central alpha-adrenoceptor stimulation involve activation of opiate receptors. As naloxone and clonidine do not appear to interact with the same receptor site, the observed functional antagonism suggests the release of an endogenous opiate by clonidine or alpha-methyldopa and the possible role of the opiate in the central control of sympathetic tone.","[naloxone, clonidine, clonidine, nalozone, alpha-methyldopa, naloxone, naloxone, clonidine, 3h-naloxone, naloxone, clonidine, 3h-dihydroergocryptine, naloxone, clonidine, clonidine, alpha-methyldopa]","[hypertensive, hypotensive, hypertensive, hypertensive]","[clonidine, nalozone, alpha-methyldopa, naloxone, naloxone, clonidine, 3h-naloxone, naloxone, clonidine, 3h-dihydroergocryptine, naloxone, clonidine, clonidine, alpha-methyldopa]","[hypertensive, hypotensive, hypertensive, hypertensive]"
1,"Intravenous administration of a single 50-mg bolus of lidocaine in a 67-year-old man resulted in profound depression of the activity of the sinoatrial and atrioventricular nodal pacemakers. The patient had no apparent associated conditions which might have predisposed him to the development of bradyarrhythmias; and, thus, this probably represented a true idiosyncrasy to lidocaine.","[lidocaine, lidocaine, lidocaine]","[cardiac asystole, depression, bradyarrhythmias]","[lidocaine, lidocaine]","[depression, bradyarrhythmias]"
2,"Suxamethonium chloride (Sch) was administered i.v. to 36 adult males at six rates: 0.25 mg s-1 to 20 mg s-1. The infusion was discontinued either when there was no muscular response to tetanic stimulation of the ulnar nerve or when Sch 120 mg was exceeded. Six additional patients received a 30-mg i.v. bolus dose. Fasciculations in six areas of the body were scored from 0 to 3 and summated as a total fasciculation score. The times to first fasciculation, twitch suppression and tetanus suppression were inversely related to the infusion rates. Fasciculations in the six areas and the total fasciculation score were related directly to the rate of infusion. Total fasciculation scores in the 30-mg bolus group and the 5-mg s-1 and 20-mg s-1 infusion groups were not significantly different.","[suxamethonium, suxamethonium chloride, sch, sch]","[fasciculations, tetanic, fasciculations, fasciculation, fasciculation, twitch, tetanus, fasciculations, fasciculation, fasciculation]","[suxamethoniumchloride, sch, sch]","[tetanic, fasciculations, fasciculation, fasciculation, twitch, tetanus, fasciculations, fasciculation, fasciculation]"


In [55]:
df_train_eval_2 = df_train_eval[['abstract', 'chemicals', 'diseases', 'predicted_chemicals', 'predicted_diseases']]
df_train_eval_2.head(5)

,abstract,chemicals,diseases,predicted_chemicals,predicted_diseases
0,"In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10(-8) to 10(-5) M, did not influence stereoselective binding of [3H]-naloxone (8 nM), and naloxone, 10(-8) to 10(-4) M, did not influence clonidine-suppressible binding of [3H]-dihydroergocryptine (1 nM). These findings indicate that in spontaneously hypertensive rats the effects of central alpha-adrenoceptor stimulation involve activation of opiate receptors. As naloxone and clonidine do not appear to interact with the same receptor site, the observed functional antagonism suggests the release of an endogenous opiate by clonidine or alpha-methyldopa and the possible role of the opiate in the central control of sympathetic tone.","[naloxone, clonidine, clonidine, nalozone, alpha-methyldopa, naloxone, naloxone, clonidine, 3h-naloxone, naloxone, clonidine, 3h-dihydroergocryptine, naloxone, clonidine, clonidine, alpha-methyldopa]","[hypertensive, hypotensive, hypertensive, hypertensive]","[clonidine, nalozone, alpha-methyldopa, naloxone, naloxone, clonidine, 3h-naloxone, naloxone, clonidine, 3h-dihydroergocryptine, naloxone, clonidine, clonidine, alpha-methyldopa]","[hypertensive, hypotensive, hypertensive, hypertensive]"
1,"Intravenous administration of a single 50-mg bolus of lidocaine in a 67-year-old man resulted in profound depression of the activity of the sinoatrial and atrioventricular nodal pacemakers. The patient had no apparent associated conditions which might have predisposed him to the development of bradyarrhythmias; and, thus, this probably represented a true idiosyncrasy to lidocaine.","[lidocaine, lidocaine, lidocaine]","[cardiac asystole, depression, bradyarrhythmias]","[lidocaine, lidocaine]","[depression, bradyarrhythmias]"
2,"Suxamethonium chloride (Sch) was administered i.v. to 36 adult males at six rates: 0.25 mg s-1 to 20 mg s-1. The infusion was discontinued either when there was no muscular response to tetanic stimulation of the ulnar nerve or when Sch 120 mg was exceeded. Six additional patients received a 30-mg i.v. bolus dose. Fasciculations in six areas of the body were scored from 0 to 3 and summated as a total fasciculation score. The times to first fasciculation, twitch suppression and tetanus suppression were inversely related to the infusion rates. Fasciculations in the six areas and the total fasciculation score were related directly to the rate of infusion. Total fasciculation scores in the 30-mg bolus group and the 5-mg s-1 and 20-mg s-1 infusion groups were not significantly different.","[suxamethonium, suxamethonium chloride, sch, sch]","[fasciculations, tetanic, fasciculations, fasciculation, fasciculation, twitch, tetanus, fasciculations, fasciculation, fasciculation]","[suxamethoniumchloride, sch, sch]","[tetanic, fasciculations, fasciculation, fasciculation, twitch, tetanus, fasciculations, fasciculation, fasciculation]"
3,"Galanthamine hydrobromide, an anticholinesterase drug capable of penetrating the blood-brain barrier, was used in a patient demonstrating central effects of scopolamine (hyoscine) overdosage. It is longer acting than physostigmine and is used in anaesthesia to reverse the non-depolarizing neuromuscular block. However, studies into the dose necessary to combating scopolamine intoxication are indicated.","[galanthamine hydrobromide, scopolamine, hyoscine, galanthamine hydrobromide, scopolamine, hyoscine, physostigmine, scopolamine]",[overdosage],"[galanthaminehydrobromide, scopolamine, hyoscine, physostigmine, scopolamine]",[]
4,"Rats with lithium-induced nephropathy 

In [53]:
# Evaluation

# Function to compute precision, recall, and F1
def calculate_metrics(true_positives, false_positives, false_negatives):
    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if (len(true_positives) + len(false_positives)) > 0 else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if (len(true_positives) + len(false_negatives)) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1

# Function to compare actual vs predicted and compute TP, FP, FN
def evaluate_chemicals_diseases(row):
    actual_chemicals = set([chem.lower().strip() for chem in row['chemicals']])
    predicted_chemicals = set([chem.lower().strip() for chem in row['predicted_chemicals']])
    
    actual_diseases = set([dis.lower().strip() for dis in row['diseases']])
    predicted_diseases = set([dis.lower().strip() for dis in row['predicted_diseases']])
    
    # Chemical metrics
    chem_tp = actual_chemicals.intersection(predicted_chemicals)
    chem_fp = predicted_chemicals.difference(actual_chemicals)
    chem_fn = actual_chemicals.difference(predicted_chemicals)
    
    # Disease metrics
    dis_tp = actual_diseases.intersection(predicted_diseases)
    dis_fp = predicted_diseases.difference(actual_diseases)
    dis_fn = actual_diseases.difference(predicted_diseases)
    
    # Calculate precision, recall, and F1 for chemicals and diseases
    chem_precision, chem_recall, chem_f1 = calculate_metrics(chem_tp, chem_fp, chem_fn)
    dis_precision, dis_recall, dis_f1 = calculate_metrics(dis_tp, dis_fp, dis_fn)
    
    return pd.Series({
        'chemical_true_positives': len(chem_tp),
        'chemical_false_positives': len(chem_fp),
        'chemical_false_negatives': len(chem_fn),
        'chemical_precision': chem_precision,
        'chemical_recall': chem_recall,
        'chemical_f1': chem_f1,
        'disease_true_positives': len(dis_tp),
        'disease_false_positives': len(dis_fp),
        'disease_false_negatives': len(dis_fn),
        'disease_precision': dis_precision,
        'disease_recall': dis_recall,
        'disease_f1': dis_f1
    })

# Apply the evaluation function to each row
df_train_eval_2[['chemical_true_positives', 'chemical_false_positives', 'chemical_false_negatives',
               'chemical_precision', 'chemical_recall', 'chemical_f1',
               'disease_true_positives', 'disease_false_positives', 'disease_false_negatives',
               'disease_precision', 'disease_recall', 'disease_f1']] = df_train_eval_2.apply(evaluate_chemicals_diseases, axis=1)

# Display the evaluation results
print(df_train_eval_2[['chemical_precision', 'chemical_recall', 'chemical_f1',
                     'disease_precision', 'disease_recall', 'disease_f1']].head())

# Calculate and display overall metrics
overall_chemical_precision = df_train_eval_2['chemical_precision'].mean()
overall_chemical_recall = df_train_eval_2['chemical_recall'].mean()
overall_chemical_f1 = df_train_eval_2['chemical_f1'].mean()

overall_disease_precision = df_train_eval_2['disease_precision'].mean()
overall_disease_recall = df_train_eval_2['disease_recall'].mean()
overall_disease_f1 = df_train_eval_2['disease_f1'].mean()

print(f"Overall Chemical Precision: {overall_chemical_precision:.4f}")
print(f"Overall Chemical Recall: {overall_chemical_recall:.4f}")
print(f"Overall Chemical F1: {overall_chemical_f1:.4f}")
print(f"Overall Disease Precision: {overall_disease_precision:.4f}")
print(f"Overall Disease Recall: {overall_disease_recall:.4f}")
print(f"Overall Disease F1: {overall_disease_f1:.4f}")


   chemical_precision  chemical_recall  chemical_f1  disease_precision  \
0                1.00         1.000000         1.00           1.000000   
1                1.00         1.000000         1.00           1.000000   
2                0.50         0.333333         0.40           1.000000   
3                0.75         0.750000         0.75           0.000000   
4                1.00         1.000000         1.00           0.666667   

   disease_recall  disease_f1  
0        1.000000    1.000000  
1        0.666667    0.800000  
2        1.000000    1.000000  
3        0.000000    0.000000  
4        0.666667    0.666667  
Overall Chemical Precision: 0.8470
Overall Chemical Recall: 0.8366
Overall Chemical F1: 0.8377
Overall Disease Precision: 0.4722
Overall Disease Recall: 0.4602
Overall Disease F1: 0.4626


/tmp/ipykernel_30/4065154045.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_eval_2[['chemical_true_positives', 'chemical_false_positives', 'chemical_false_negatives',
/tmp/ipykernel_30/4065154045.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_eval_2[['chemical_true_positives', 'chemical_false_positives', 'chemical_false_negatives',
/tmp/ipykernel_30/4065154045.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

- True Positives (TP): Correctly predicted entities.
- False Positives (FP): Predicted entities that don’t appear in the actual entities.
- False Negatives (FN): Actual entities that were not predicted.